# Лабораторна робота №6
## Завдання 1

1. Завантаження та підготовка даних:
    - Виберіть вбудований датасет для задачі регресії з бібліотеки scikit-learn
(наприклад, Boston Housing, California Housing) або будь-який датасет
для регресії.
    - Завантажте дані та розділіть їх на навчальну та тестову вибірки
(рекомендується використовувати співвідношення 80/20 або 70/30).
2. Побудова регресійної моделі:
    - Побудуйте базову лінійну регресійну модель на основі навчальної вибірки.
    - Оцініть точність моделі на тестовій вибірці та визначте її продуктивність,
використовуючи метрики якості регресії (наприклад, mean_squared_error,
R^2).
3. Аналіз перенавчання:
    - Перевірте наявність перенавчання, порівнюючи результати на навчальній та
тестовій вибірках. Якщо модель показує значно кращі результати на
навчальних даних, це може свідчити про перенавчання.
4. Запобігання перенавчанню:
    - Введіть регуляризацію: використайте моделі з регуляризацією, такі як Ridge
Regression або Lasso Regression.
    - Перевірте роботу моделі з різними значеннями параметра регуляризації та
виберіть оптимальне значення.
    - Застосуйте крос-валідацію для більш стабільної оцінки продуктивності
моделі.
5. Оцінка результатів:
    - Оцініть кінцеву продуктивність моделі на тестовій вибірці та порівняйте її з
базовою моделлю.
    - Проаналізуйте, як введення регуляризації та використання крос-валідації
вплинули на якість моделі.
6. Візуалізація та висновки:
    - Побудуйте графіки, що демонструють залежність помилки моделі від
гіперпараметрів регуляризації.
    - Підготуйте звіт з висновками щодо ефективності моделі, заходів щодо
запобігання перенавчанню та загальних результатів.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import datasets

In [7]:
data = datasets.fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

print(data.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [8]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


In [9]:
X = df.copy()
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_test)

In [12]:
from sklearn.metrics import mean_squared_error, r2_score

print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R2 Score:', r2_score(y_test, y_pred))
print('Accuracy:', linear.score(X_test, y_test))

Mean Squared Error: 0.5558915986952444
R2 Score: 0.5757877060324508
Accuracy: 0.5757877060324508


In [13]:
# Test overfitting
print('Accuracy:', linear.score(X_train, y_train))

Accuracy: 0.6125511913966952


In [19]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, cross_val_score

ridge = Ridge()
lasso = Lasso()

parameters = {'alpha': np.logspace(-5, 2, 8)}

ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X_train, y_train)

lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=5)
lasso_regressor.fit(X_train, y_train)

print(f'Ridge Best Parameters: {ridge_regressor.best_params_}')
print(f'Lasso Best Parameters: {lasso_regressor.best_params_}')

ridge_best = Ridge(**ridge_regressor.best_params_)
lasso_best = Lasso(**lasso_regressor.best_params_)

ridge_cv_scores = cross_val_score(ridge_best, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
lasso_cv_scores = cross_val_score(lasso_best, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

print('Ridge CV Mean Squared Error:', -ridge_cv_scores.mean())
print('Lasso CV Mean Squared Error:', -lasso_cv_scores.mean())

ridge_best.fit(X_train, y_train)
lasso_best.fit(X_train, y_train)

y_pred_ridge = ridge_best.predict(X_test)
y_pred_lasso = lasso_best.predict(X_test)

print('Ridge Mean Squared Error:', mean_squared_error(y_test, y_pred_ridge))
print('Ridge R2 Score:', r2_score(y_test, y_pred_ridge))
print('Ridge Accuracy:', ridge_best.score(X_test, y_test))
print('Ridge Accuracy with training data:', ridge_best.score(X_train, y_train))

print('Lasso Mean Squared Error:', mean_squared_error(y_test, y_pred_lasso))
print('Lasso R2 Score:', r2_score(y_test, y_pred_lasso))
print('Lasso Accuracy:', lasso_best.score(X_test, y_test))
print('Lasso Accuracy with training data:', lasso_best.score(X_train, y_train))



Ridge Best Parameters: {'alpha': np.float64(1.0)}
Lasso Best Parameters: {'alpha': np.float64(0.001)}
Ridge CV Mean Squared Error: 0.51926515697919
Lasso CV Mean Squared Error: 0.5192487792639193
Ridge Mean Squared Error: 0.5558512007367515
Ridge R2 Score: 0.5758185345441319
Ridge Accuracy: 0.5758185345441319
Ridge Accuracy with training data: 0.612551119993334
Lasso Mean Squared Error: 0.5544062174455686
Lasso R2 Score: 0.5769212309657656
Lasso Accuracy: 0.5769212309657656
Lasso Accuracy with training data: 0.6125069391599488


In [21]:
# Compare base linear regression with Ridge and Lasso
# Using cross validation
linear_cv_scores = cross_val_score(linear, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
ridge_cv_scores = cross_val_score(ridge_best, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
lasso_cv_scores = cross_val_score(lasso_best, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

print('Linear CV Mean Squared Error:', -linear_cv_scores.mean())
print('Ridge CV Mean Squared Error: ', -ridge_cv_scores.mean())
print('Lasso CV Mean Squared Error: ', -lasso_cv_scores.mean())

Linear CV Mean Squared Error: 0.5192652011433679
Ridge CV Mean Squared Error:  0.51926515697919
Lasso CV Mean Squared Error:  0.5192487792639193


У випадку з даним датасетом перенавчення не є великою проблемою навіть з лінійною регресією. А тому навіть з регуляризацією L1 та L2, результати дужі схожі як з тестовиими так і з тренувальними даними.